# Playing Around with pvlib

In [2]:
#IMPORTS
import pandas as pd
import os
import numpy as np
import pysolar
import pvlib
import datetime
import pytz
import math
from dateutil import tz

print(os.listdir())

['.irradience_fitter.ipynb.icloud', 'playing_with_pvlib.ipynb', 'statistical_modeling.py', '.ipynb_checkpoints', '.statisical_irradience_modeling 3.ipynb.icloud']


In [17]:
_Curr_Dir = os.path.dirname(os.getcwd())
_Testing_Data_Dir = os.path.join(_Curr_Dir, 'Testing_Data')
print(os.listdir(_Testing_Data_Dir))

['cloud_output_darksky_2009.csv', 'cloud_output_darksky_2020.csv', 'cloud_output_darksky_2008.csv', 'data_munger.py', 'cloud_output_darksky_2018.csv', 'cloud_output_darksky_2019.csv', 'psm_testing_data2018_copy.csv', 'Charlottesville_psm_2018_cloud_output_darksky_2018.csv', 'psm_testing_data2009.csv', 'psm_testing_data2008.csv', 'psm_testing_data2018.csv', 'Austin_psm_2018.csv', 'psm_testing_data2017.csv', 'psm_testing_data2016.csv', 'psm_testing_data2014.csv', 'Austin_psm_2018_cloud_output_darksky_2018.csv', 'psm_testing_data2015.csv', 'psm_testing_data2011.csv', 'psm_testing_data2010.csv', 'psm_testing_data2012.csv', 'psm_testing_data2007.csv', 'psm_testing_data2013.csv', 'cloud_output_darksky_2014.csv', 'cloud_output_darksky_2015.csv', 'cloud_output_darksky_2017.csv', 'cloud_output_darksky_2016.csv', 'cloud_output_darksky_2012.csv', 'Charlottesville_psm_2018.csv', 'cloud_output_darksky_2013.csv', 'cloud_output_darksky_2007.csv', 'cloud_output_darksky_2011.csv', 'cloud_output_darksky

In [18]:
def get_dni(ghi, lat, lon, day):
    # pvlib.irradiance.disc(ghi, solar_zenith, datetime_or_doy, pressure=101325, min_cos_zenith=0.065, max_zenith=87, max_airmass=12) 
    utcTime, date = _make_good_date(day[0], day[1], day[2], day[3], day[4])
    solar_altitude = pysolar.solar.get_altitude(lat,lon,date)
    solar_zenith = 90 - solar_altitude
    DNI = pvlib.irradiance.disc(ghi, solar_zenith, date)
    return DNI

In [19]:
def _make_good_date(year,month,day,hour,minute):
    date = datetime.datetime(year, month, day, hour, minute)
    date = pytz.timezone('US/Eastern').localize(date)
    utcTime = date.astimezone(pytz.timezone('UTC'))
    return utcTime, date

In [20]:
def make_df_nonzero(df):
    df_no_zero = df.loc[df['DHI']!= 0 ]
    assert len(df_no_zero['DHI'])==len(df_no_zero['GHI']), "Some DHI 0s where GHI is not 0"
    print(df_no_zero)
    return df_no_zero 

In [41]:
print(_make_good_date(2018,8,20,21,30))
dni_pred = get_dni(648, 43.85, -99.5, [2018,8,20,22,30])['dni']
print(dni_pred)

(datetime.datetime(2018, 8, 21, 1, 30, tzinfo=<UTC>), datetime.datetime(2018, 8, 20, 21, 30, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>))
0.0


In [21]:
# date = datetime.datetime(2018, 1, 22, 15, 30)
# date = pytz.timezone('US/Eastern').localize(date)
# print(date)
# utcTime = date.astimezone(pytz.timezone('UTC'))
# print(utcTime)

In [22]:
#Get 2018 Data
df1_2018 = pd.read_csv(os.path.join(_Testing_Data_Dir, 'psm_testing_data2018.csv'))
df1 = df1_2018[['Month','Day','Hour','Minute','GHI', 'DNI', 'DHI']]
df1 = make_df_nonzero(df1)
print(df1)
# print(df1['Year'])

      Month  Day  Hour  Minute  GHI  DNI  DHI
14        1    1    14      30   14   50   12
15        1    1    15      30  106  301   53
16        1    1    16      30  274  742   60
17        1    1    17      30  376  860   64
18        1    1    18      30  440  976   56
...     ...  ...   ...     ...  ...  ...  ...
8754     12   31    18      30  382  812   64
8755     12   31    19      30  316  569  101
8756     12   31    20      30  257  525   89
8757     12   31    21      30  154  385   69
8758     12   31    22      30   50  191   33

[4494 rows x 7 columns]
      Month  Day  Hour  Minute  GHI  DNI  DHI
14        1    1    14      30   14   50   12
15        1    1    15      30  106  301   53
16        1    1    16      30  274  742   60
17        1    1    17      30  376  860   64
18        1    1    18      30  440  976   56
...     ...  ...   ...     ...  ...  ...  ...
8754     12   31    18      30  382  812   64
8755     12   31    19      30  316  569  101
8756     

In [32]:
preds = []
resids = []
actuals = []
for i in range(len(df1)):
    lat = 43.85
    lon = -99.5
    year = 2018
    month = (df1.iloc[i]['Month'])
    day = df1.iloc[i]['Day']
    hour = df1.iloc[i]['Hour']
    minute = df1.iloc[i]['Minute']
    date = [year, month, day, hour, minute]
    ghi = df1.iloc[i]['GHI']
    dni_actual = df1.iloc[i]['DNI']
    dni_pred = get_dni(ghi, lat, lon, date)['dni']
    preds.append(dni_pred)
    resids.append(abs(dni_actual-dni_pred))
    actuals.append(dni_actual)
    
    
    
    
    

In [33]:
print(resids)

[50.0, 301.0, 14.012853883505272, 503.19519401311175, 976.0, 967.0, 926.0, 830.0, 597.0, 123.0, 562.4085058514293, 17.948969511252812, 15.68464170470969, 54.0, 78.0, 196.0, 239.0, 46.0, 18.0, 116.0, 223.05873091457147, 159.33020444987517, 531.0, 816.0, 295.0, 212.0, 246.0, 4.0, 195.0, 110.01769572554849, 454.2488019321643, 694.0, 604.0, 803.0, 437.0, 401.0, 24.0, 206.0, 56.009874507307075, 88.07944339991735, 657.0, 653.0, 630.0, 171.0, 96.0, 110.0, 485.5758670489903, 4.0035688992228415, 255.8298456875425, 591.0, 585.0, 91.0, 480.0, 260.0, 140.0, 390.6805046017161, 120.73831898069409, 417.5073527968188, 406.0, 727.0, 615.0, 544.0, 185.0, 109.0, 507.40728529773605, 235.36559945610657, 146.11909366100986, 642.0, 426.0, 597.0, 553.0, 368.0, 124.0, 445.8729825361809, 157.37720804922333, 242.6720822518883, 701.0, 594.0, 374.0, 385.0, 390.0, 96.0, 337.8821460599327, 94.29724366845369, 252.82672422034625, 187.0, 140.0, 167.0, 0.0, 0.0, 25.0, 0.0, 187.60270365530812, 435.62961127413575, 669.0, 

In [34]:
print(np.mean(resids))

284.3058715526506


In [35]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error, median_absolute_error


In [36]:
actuals = np.array(actuals)
preds = np.array(preds)
print("mean absolute error is ", mean_absolute_error(actuals, preds))
print("MAPE is ", np.mean(np.abs((actuals - preds) / actuals)) * 100)


mean absolute error is  284.3058715526506
MAPE is  nan


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [37]:
for i in range(20):
    print(preds[i], actuals[i], resids[i])

0.0 50 50.0
0.0 301 301.0
727.9871461164947 742 14.012853883505272
356.80480598688825 860 503.19519401311175
0.0 976 976.0
0.0 967 967.0
0.0 926 926.0
0.0 830 830.0
0.0 597 597.0
0.0 123 123.0
70.59149414857073 633 562.4085058514293
733.9489695112528 716 17.948969511252812
362.6846417047097 347 15.68464170470969
0.0 54 54.0
0.0 78 78.0
0.0 196 196.0
0.0 239 239.0
0.0 46 46.0
0.0 18 18.0
0.0 116 116.0


In [42]:
#For many predictions, that are NOT 0, they are pretty close!